In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import duckdb as db
import requests

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import plotly.express as px

data loading (3 csv files found on kaggle)
Ísar - We should retrieve this data via API as to fulfill the retrieval part of the assignment

In [18]:
df_cancer_reg = pd.read_csv("cancer_reg.csv") # where is the link to this kaggle dataset?
df_avg_household_size = pd.read_csv("avg-household-size.csv") # where is the link to this kaggle dataset?
df_life_expectancy = pd.read_csv("life_expectancy_data.csv") # where is the link to this kaggle dataset?

In [ ]:
conn = db.connect(database='cancer_data.ddb', read_only=False)
conn.execute("CREATE TABLE IF NOT EXISTS Life_Expectancy_Data AS SELECT * FROM df_life_expectancy")
conn.close()

In [20]:
conn = db.connect(database='cancer_data.ddb', read_only=True)
df_cancer_reg = conn.execute("SELECT * FROM cancer_reg").df()
conn.close()

In [21]:
print(f"""
df_avg_household_size.shape: {df_avg_household_size.shape}
df_cancer_reg.shape: {df_cancer_reg.shape}
df_life_expectancy.shape: {df_life_expectancy.shape}
""")


df_avg_household_size.shape: (3220, 4)
df_cancer_reg.shape: (3047, 33)
df_life_expectancy.shape: (2938, 22)



In [51]:
df_life_expectancy.isnull().sum(), df_life_expectancy.shape

(Country                              0
 Year                                 0
 Status                               0
 Life expectancy                     10
 Adult Mortality                     10
 infant deaths                        0
 Alcohol                            194
 percentage expenditure               0
 Hepatitis B                        553
 Measles                              0
  BMI                                34
 under-five deaths                    0
 Polio                               19
 Total expenditure                  226
 Diphtheria                          19
  HIV/AIDS                            0
 GDP                                448
 Population                         652
  thinness  1-19 years               34
  thinness 5-9 years                 34
 Income composition of resources    167
 Schooling                          163
 dtype: int64,
 (2938, 22))

In [49]:
df_life_expectancy[df_life_expectancy.isnull().any(axis=1)].sort_values(by=['Country','Year'], ascending=False)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
2922,Zimbabwe,2015,Developing,67.0,336.0,22,NaN,0.000000,87.0,0,...,88.0,NaN,87.0,6.2,118.693830,15777451.0,5.6,5.5,0.507,10.3
2906,Zambia,2015,Developing,61.8,33.0,27,NaN,0.000000,9.0,9,...,9.0,NaN,9.0,4.1,1313.889646,161587.0,6.3,6.1,0.576,12.5
2917,Zambia,2004,Developing,47.9,578.0,36,2.46,8.369852,NaN,35,...,84.0,7.33,83.0,17.6,53.277222,11731746.0,7.2,7.1,0.456,10.5
2918,Zambia,2003,Developing,46.4,64.0,39,2.33,65.789974,NaN,881,...,85.0,8.18,83.0,18.2,429.158343,11421984.0,7.3,7.2,0.443,10.2
2919,Zambia,2002,Developing,45.5,69.0,41,2.44,54.043480,NaN,25036,...,85.0,6.93,84.0,18.4,377.135244,111249.0,7.4,7.3,0.433,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,Algeria,2015,Developing,75.6,19.0,21,NaN,0.000000,95.0,63,...,95.0,NaN,95.0,0.1,4132.762920,39871528.0,6.0,5.8,0.743,14.4
44,Algeria,2003,Developing,71.7,146.0,20,0.34,25.018523,NaN,15374,...,87.0,3.60,87.0,0.1,294.335560,3243514.0,6.3,6.1,0.663,11.5
45,Algeria,2002,Developing,71.6,145.0,20,0.36,148.511984,NaN,5862,...,86.0,3.73,86.0,0.1,1774.336730,3199546.0,6.3,6.2,0.653,11.1
46,Algeria,2001,Developing,71.4,145.0,20,0.23,147.986071,NaN,2686,...,89.0,3.84,89.0,0.1,1732.857979,31592153.0,6.4,6.3,0.644,10.9


In [22]:
df_avg_household_size.head(10)

,statefips,countyfips,avghouseholdsize,geography
0,2,13,2.43,"Aleutians East Borough, Alaska"
1,2,16,3.59,"Aleutians West Census Area, Alaska"
2,2,20,2.77,"Anchorage Municipality, Alaska"
3,2,50,3.86,"Bethel Census Area, Alaska"
4,2,60,2.50,"Bristol Bay Borough, Alaska"
5,2,68,2.34,"Denali Borough, Alaska"
6,2,70,3.56,"Dillingham Census Area, Alaska"
7,2,90,2.69,"Fairbanks North Star Borough, Alaska"
8,2,100,2.12,"Haines Borough, Alaska"
9,2,105,2.26,"Hoonah-Angoon Census Area, Alaska"


In [23]:
df_cancer_reg.head(10)

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",39.3,...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",45.0,...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657
5,428.0,152,176.0,505.4,52313,61023,15.6,180.259902,"(51046.4, 54545.6]",45.4,...,38.8,32.6,43.2,20.2,84.882631,1.653205,1.538057,3.314635,51.220360,4.964476
6,250.0,97,175.9,461.8,37782,41516,23.2,0.000000,"(37413.8, 40362.7]",42.6,...,35.0,28.3,46.4,28.7,75.106455,0.616955,0.866157,8.356721,51.013900,4.204317
7,146.0,71,183.6,404.0,40189,20848,17.8,0.000000,"(37413.8, 40362.7]",51.7,...,33.1,25.9,50.9,24.1,89.406636,0.305159,1.889077,2.286268,48.967033,5.889179
8,88.0,36,190.5,459.4,42579,13088,22.3,0.000000,"(40362.7, 42724.4]",49.3,...,37.8,29.9,48.1,26.6,91.787477,0.185071,0.208205,0.616903,53.446998,5.587583
9,4025.0,1380,177.8,510.9,60397,843954,13.1,427.748432,"(54545.6, 61494.5]",35.8,...,NaN,44.4,31.4,16.5,74.729668,6.710854,6.041472,2.699184,50.063573,5.533430


In [24]:
df_life_expectancy.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [25]:
"""dataset_info = {
    "df_avg_household_size": {
        "Rows": df_avg_household_size.shape[0],
        "Columns": df_avg_household_size.shape[1],
        "Column Names": df_avg_household_size.columns.tolist(),
    },
    "df_cancer_reg": {
        "Rows": df_cancer_reg.shape[0],
        "Columns": df_cancer_reg.shape[1],
        "Column Names": df_cancer_reg.columns.tolist(),
    },
    "df_life_expectancy": {
        "Rows": df_life_expectancy.shape[0],
        "Columns": df_life_expectancy.shape[1],
        "Column Names": df_life_expectancy.columns.tolist(),
    },
}
"""

'dataset_info = {\n    "df_avg_household_size": {\n        "Rows": df_avg_household_size.shape[0],\n        "Columns": df_avg_household_size.shape[1],\n        "Column Names": df_avg_household_size.columns.tolist(),\n    },\n    "df_cancer_reg": {\n        "Rows": df_cancer_reg.shape[0],\n        "Columns": df_cancer_reg.shape[1],\n        "Column Names": df_cancer_reg.columns.tolist(),\n    },\n    "df_life_expectancy": {\n        "Rows": df_life_expectancy.shape[0],\n        "Columns": df_life_expectancy.shape[1],\n        "Column Names": df_life_expectancy.columns.tolist(),\n    },\n}\n'

In [26]:
df_avg_household_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   statefips         3220 non-null   int64  
 1   countyfips        3220 non-null   int64  
 2   avghouseholdsize  3220 non-null   float64
 3   geography         3220 non-null   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 100.8+ KB


In [27]:
df_cancer_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avganncount              3047 non-null   float64
 1   avgdeathsperyear         3047 non-null   int64  
 2   target_deathrate         3047 non-null   float64
 3   incidencerate            3047 non-null   float64
 4   medincome                3047 non-null   int64  
 5   popest2015               3047 non-null   int64  
 6   povertypercent           3047 non-null   float64
 7   studypercap              3047 non-null   float64
 8   binnedinc                3047 non-null   object 
 9   medianage                3047 non-null   float64
 10  medianagemale            3047 non-null   float64
 11  medianagefemale          3047 non-null   float64
 12  geography                3047 non-null   object 
 13  percentmarried           3047 non-null   float64
 14  pctnohs18_24            

In [28]:
df_life_expectancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

**DATAVIZ**

dataviz for first csv (avg-household-size) (not so relevant in our case).

In [29]:
# Visualization 1: Distribution of Average Household Size
fig1 = px.histogram(
    df_avg_household_size, 
    x="avghouseholdsize", 
    nbins=30, 
    title="Distribution of Average Household Size",
    labels={"avghouseholdsize": "Average Household Size"},
    opacity=0.75
)
fig1.show()

AVG household only in the US, not relevant data since its only in the US

In [30]:
df_sorted = df_avg_household_size.sort_values(by='avghouseholdsize', ascending=False).head(20)


In [31]:
fig1 = px.bar(df_sorted, 
             x='geography', 
             y='avghouseholdsize', 
             title='Top 20 Counties by Average Household Size',
             labels={'geography': 'County', 'avghouseholdsize': 'Average Household Size'},
             color='avghouseholdsize', 
             color_continuous_scale='Blues')
fig1.show()

In [32]:
fig4 = px.box(df_avg_household_size, 
              x='statefips', 
              y='avghouseholdsize',
              title='Household Size Distribution by State',
              labels={'statefips': 'State FIPS', 'avghouseholdsize': 'Average Household Size'},
              color='statefips')
fig4.show()

**SECOND CSV : Cancer analysis**


In [33]:
df_sorted_cancer = df_cancer_reg.sort_values(by='incidencerate', ascending=False).head(20)

In [34]:
# Scatter plot - Median income vs Cancer Incidence Rate
fig4 = px.scatter(df_cancer_reg, 
                  x='medincome', 
                  y='incidencerate', 
                  title='Cancer Incidence Rate vs Median Income',
                  labels={'medincome': 'Median Income', 'incidencerate': 'Cancer Incidence Rate'},
                  color='incidencerate',
                  color_continuous_scale='Viridis')
fig4.show()


In [35]:
fig5 = px.box(df_cancer_reg, 
              x='binnedinc', 
              y='target_deathrate', 
              title='Cancer Death Rate Distribution by Income Level',
              labels={'binnedinc': 'Income Bins', 'target_deathrate': 'Cancer Death Rate'},
              color='binnedinc')
fig5.show()


In [36]:
# Line chart - Average deaths per year vs median age (focused on 0-100)
fig7 = px.line(df_cancer_reg[df_cancer_reg['medianage'] <= 100].sort_values(by='medianage'), 
               x='medianage', 
               y='avgdeathsperyear', 
               title='Average Deaths per Year vs Median Age (0-100)',
               labels={'medianage': 'Median Age', 'avgdeathsperyear': 'Average Deaths per Year'},
               markers=True)
fig7.show()


In [37]:
fig6 = px.histogram(df_cancer_reg, 
                    x='povertypercent', 
                    title='Distribution of Poverty Percentage with KDE',
                    nbins=30, 
                    marginal="violin", 
                    color_discrete_sequence=['orange'])
fig6.show()


**NOW WE MIX BOTH DATAFRAME** (life expectancy and cancer_reg)

In [ ]:
fig2 = px.scatter(df_life_expectancy, x="GDP", y="Life expectancy ", 
                  title="Life Expectancy vs. GDP",
                  labels={"GDP": "Gross Domestic Product", "Life expectancy ": "Life Expectancy"},
                  trendline="ols")
fig2.show()

In [ ]:
fig3 = px.scatter(df_cancer_reg, x="pctpubliccoverage", y="target_deathrate", 
                  title="Public Health Coverage vs. Cancer Death Rate",
                  labels={"pctpubliccoverage": "Public Health Coverage (%)", "target_deathrate": "Cancer Death Rate"},
                  color="pctprivatecoverage")
fig3.show()


In [ ]:
# Alcohol Consumption vs. Life Expectancy
fig4 = px.scatter(df_life_expectancy, x="Alcohol", y="Life expectancy ", 
                  title="Alcohol Consumption vs. Life Expectancy",
                  labels={"Alcohol": "Alcohol Consumption (liters per capita)", "Life expectancy ": "Life Expectancy"},
                  trendline="ols")
fig4.show()

In [ ]:
fig5 = px.scatter(df_cancer_reg, x="povertypercent", y="incidencerate", 
                  title="Poverty Rate vs. Cancer Incidence",
                  labels={"povertypercent": "Poverty Rate (%)", "incidencerate": "Cancer Incidence Rate"},
                  trendline="ols")

In [ ]:
fig5.show()

let's merge everything

In [ ]:
merged_df = df_avg_household_size.merge(df_cancer_reg, left_on='geography', right_on='geography', how='inner')


In [ ]:
fig8 = px.scatter(merged_df, 
                  x='avghouseholdsize', 
                  y='incidencerate', 
                  title='Household Size vs Cancer Incidence Rate',
                  labels={'avghouseholdsize': 'Average Household Size', 'incidencerate': 'Cancer Incidence Rate'},
                  color='incidencerate',
                  color_continuous_scale='Plasma')
fig8.show()

In [ ]:
fig9 = px.box(merged_df, 
              x='avghouseholdsize', 
              y='target_deathrate', 
              title='Cancer Death Rate vs Household Size',
              labels={'avghouseholdsize': 'Average Household Size', 'target_deathrate': 'Cancer Death Rate'},
              color='avghouseholdsize')
fig9.show()

In [ ]:
filtered_cancer_df = df_cancer_reg[df_cancer_reg["medianage"] <= 80]

fig6 = px.scatter(filtered_cancer_df, x="medianage", y="target_deathrate", 
                  title="Cancer Death Rate vs. Median Age (Up to 80 Years)",
                  labels={"medianage": "Median Age", "target_deathrate": "Cancer Death Rate"},
                  trendline="ols")

# Show the figure
fig6.show()

